In [6]:
import numpy as np
from tensorflow.keras.layers import Dense, Activation
from tensorflow.keras.layers import SimpleRNN, LSTM, GRU
from tensorflow.keras.models import Sequential

Abba_money = ["I work all night, I work all day, to pay the bills I have to pay", 
              "Ain't it sad And still there never seems to be a single penny left for me", 
              "That's too bad In my dreams I have a plan",
              "If I got me a wealthy man I wouldn't have to work at all, I'd fool around and have a ball...", 
              "Money, money, money Must be funny In the rich man's world",
              "Money, money, money Always sunny In the rich man's world", 
              "Aha-ahaaa All the things I could do If I had a little money It's a rich man's world",
              "A man like that is hard to find but I can't get him off my mind",
              "Ain't it sad And if he happens to be free I bet he wouldn't fancy me",
              "That's too bad So I must leave, I'll have to go To Las Vegas or Monaco And win a fortune in a game, my life will never be the same...",
              "Money, money, money Must be funny In the rich man's world",
              "Money, money, money Always sunny In the rich man's world",
              "Aha-ahaaa All the things I could do If I had a little money It's a rich man's world",
              "Money, money, money Must be funny In the rich man's world",
              "Money, money, money Always sunny In the rich man's world",
              "Aha-ahaaa All the things I could do If I had a little money It's a rich man's world It's a rich man's world"]

_in = Abba_money
# построчное чтение из примера с текстом 
lines = [] 
for line in _in:
    line = line.strip().lower() #.decode("ascii", "ignore")
    if len(line) == 0:
        continue
    lines.append(line)
text = " ".join(lines)
chars = set([c for c in text])
nb_chars = len(chars)


# создание индекса символов и reverse mapping чтобы передвигаться между значениями numerical
# ID and a specific character. The numerical ID will correspond to a column
# ID и определенный символ. Numerical ID будет соответсвовать колонке
# число при использовании one-hot кодировки для представление входов символов
char2index = {c: i for i, c in enumerate(chars)}
index2char = {i: c for i, c in enumerate(chars)}

# для удобства выберете фиксированную длину последовательность 10 символов 
SEQLEN, STEP = 10, 1
input_chars, label_chars = [], []

# конвертация data в серии разных SEQLEN-length субпоследовательностей
for i in range(0, len(text) - SEQLEN, STEP):
    input_chars.append(text[i: i + SEQLEN])
    label_chars.append(text[i + SEQLEN])


# Вычисление one-hot encoding входных последовательностей X и следующего символа (the label) y

X = np.zeros((len(input_chars), SEQLEN, nb_chars), dtype=np.bool)
y = np.zeros((len(input_chars), nb_chars), dtype=np.bool)
for i, input_char in enumerate(input_chars):
    for j, ch in enumerate(input_char):
        X[i, j, char2index[ch]] = 1
    y[i, char2index[label_chars[i]]] = 1


# установка ряда метапамертров  для нейронной сети и процесса тренировки
BATCH_SIZE, HIDDEN_SIZE = 128, 128
NUM_ITERATIONS = 1 # 25 должно быть достаточно
NUM_EPOCHS_PER_ITERATION = 1
NUM_PREDS_PER_EPOCH = 100


# Create a super simple recurrent neural network. There is one recurrent
# layer that produces an embedding of size HIDDEN_SIZE from the one-hot
# encoded input layer. This is followed by a Dense fully-connected layer
# across the set of possible next characters, which is converted to a
# probability score via a standard softmax activation with a multi-class
# cross-entropy loss function linking the prediction to the one-hot
# encoding character label.

'''
Создание очень простой рекуррентной нейронной сети. В ней будет один реккурентный закодированный входной слой. За ним последует полносвязный слой связанный с набором возможных следующих символов, которые конвертированы в вероятностные результаты через стандартную softmax активацию с multi-class cross-encoding loss функцию ссылающуются на предсказание one-hot encoding лейбл символа
'''

model = Sequential()
model.add(
    GRU(  # вы можете изменить эту часть на LSTM или SimpleRNN, чтобы попробовать альтернативы
        HIDDEN_SIZE,
        return_sequences=False,
        input_shape=(SEQLEN, nb_chars),
        unroll=True
    )
)
model.add(Dense(nb_chars))
model.add(Activation("softmax"))
model.compile(loss="categorical_crossentropy", optimizer="rmsprop")


# выполнение серий тренировочных и демонстрационных итераций 
for iteration in range(NUM_ITERATIONS):

    # для каждой итерации запуск передачи данных в модель 
    print("=" * 50)
    print("Итерация #: %d" % (iteration))
    model.fit(X, y, batch_size=BATCH_SIZE, epochs=NUM_EPOCHS_PER_ITERATION)

    # Select a random example input sequence.
    test_idx = np.random.randint(len(input_chars))
    test_chars = input_chars[test_idx]

    # для числа шагов предсказаний использование текущей тренируемой модели 
    # конструирование one-hot encoding для тестирования input и добавление предсказания.
    print("Генерация из посева: %s" % (test_chars))
    print(test_chars, end="")
    for i in range(NUM_PREDS_PER_EPOCH):

        # здесь one-hot encoding.
        X_test = np.zeros((1, SEQLEN, nb_chars))
        for j, ch in enumerate(test_chars):
            X_test[0, j, char2index[ch]] = 1

        # осуществление предсказания с помощью текущей модели.
        pred = model.predict(X_test, verbose=0)[0]
        y_pred = index2char[np.argmax(pred)]

        # вывод предсказания добавленного к тестовому примеру 
        print("pred:", y_pred, end="")

        # инкрементация тестового примера содержащего предсказание
        test_chars = test_chars[1:] + y_pred
print(test_chars)


Итерация #: 0
9/9 [==============================] - 0s 5ms/step - loss: 3.1778
Генерация из посева: ey always 
ey always pred:  pred:  pred:  pred:  pred:  pred:  pred:  pred:  pred:  pred:  pred:  pred:  pred:  pred:  pred:  pred:  pred:  pred:  pred:  pred:  pred:  pred:  pred:  pred:  pred:  pred:  pred:  pred:  pred:  pred:  pred:  pred:  pred:  pred:  pred:  pred:  pred:  pred:  pred:  pred:  pred:  pred:  pred:  pred:  pred:  pred:  pred:  pred:  pred:  pred:  pred:  pred:  pred:  pred:  pred:  pred:  pred:  pred:  pred:  pred:  pred:  pred:  pred:  pred:  pred:  pred:  pred:  pred:  pred:  pred:  pred:  pred:  pred:  pred:  pred:  pred:  pred:  pred:  pred:  pred:  pred:  pred:  pred:  pred:  pred:  pred:  pred:  pred:  pred:  pred:  pred:  pred:  pred:  pred:  pred:  pred:  pred:  pred:  pred:  pred:            
